In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from numpy import full

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
)

template_ask_movie_title = ChatPromptTemplate.from_messages(
    [
        ("system","영화제목을 묻는 질문에 단답형으로 대답하세요.",),
        ("human", "{question}"),
    ]
)

examples = [
    {
        "movie_title": "닥터 스트레인지",
        "answer": """
        감독: 스콧 데릭슨
        국가: 미국
        출연진: 베네딕트 컴버배치, 레이첼 맥아담스, 틸다 스윈튼...
        예산: 1억 6500만 달러
        흥행 수익: 1억 2,200만 달러
        장르: 액션, 모험, 판타지
        시놉시스: 불의의 사고로 절망에 빠진 천재 외과의사 ‘닥터 스트레인지(베네딕트 컴버배치)’. 마지막 희망을 걸고 찾아 간 곳에서 ‘에인션트 원(틸다 스윈튼)’을 만나 세상을 구원할 강력한 능력을 얻게 되면서, 모든 것을 초월한 최강의 히어로로 거듭나는데...
        """,
    },
    {
        "movie_title": "괴물",
        "answer": """
        감독: 고로에다 히로카즈
        국가: 일본
        출연진: 안도 사쿠라, 나가야마 에이타, 구로카와 소야, 히이라기 히나타...
        흥행 수익: 2천만 달러
        장르: 드라마, 스릴러
        시놉시스: “우리 동네에는 괴물이 산다” 싱글맘 사오리(안도 사쿠라)는 아들 미나토(쿠로카와 소야)의 행동에서 이상 기운을 감지한다. 용기를 내 찾아간 학교에서 상담을 진행한 날 이후 선생님과 학생들의 분위기가 심상치 않게 흐르기 시작하고. “괴물은 누구인가?” 한편 사오리는 친구들로부터 따돌림을 당하고 있는 미나토의 친구 요리(히이라기 히나타)의 존재를 알게 되고 자신이 아는 아들의 모습과 사람들이 아는 아들의 모습이 다르다는 사실을 어렴풋이 깨닫는데… 태풍이 몰아치던 어느 날, 아무도 몰랐던 진실이 드러난다.
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","영화 '{movie_title}'에 대해서 알고 있어?"),
        ("ai","{answer}"),
    ]
)

example_prompt_fewshot = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 전문가야. 너는 모른다고 답하지 않아. 질문에 대해 성실하게 답변해."),
        example_prompt_fewshot,
        ("human", "영화 '{movie_title}'에 대해서 알고 있어?"),
    ]
)

chain_ask_movie = template_ask_movie_title | chat
chain_explain_movie = final_prompt | chat
full_chain = {"movie_title": chain_ask_movie} | chain_explain_movie

result = full_chain.invoke({"question": '너가 자세하게 알고 있는 영화 제목을 하나 알려줘.'})
print(result.content)

죄송하지만, 제가 제대로된 정보를 제공하기 위해서는 영화 제목이 올바르게 입력되어야 합니다. "인셉션"에 대해 말씀드리겠습니다. 

        감독: 크리스토퍼 놀란
        국가: 미국
        출연진: 레오나르도 디카프리오, 조셉 고든 레빗, 엘렌 페이지, 톰 하디...
        예산: 1억 6000만 달러
        흥행 수익: 8억 2000만 달러
        장르: 액션, SF, 스릴러
        시놉시스: 꿈 속으로 침투해 정보를 훔치는 특수한 능력을 가진 도둑 코브(레오나르도 디카프리오)는 한 번의 마지막 작전을 성공시키면 자신의 죄를 사면해줄 것을 약속받는다. 그 작전은 인셉션, 즉 타인의 마음 속에 아이디어를 심는 것이다. 그러나 이번 작전은 그 어느 때보다 위험하고 복잡하며, 코브와 그의 팀은 현실과 꿈의 경계를 넘나들며 예상치 못한 위기에 빠지게 되는데...
